![Title](https://upload.wikimedia.org/wikipedia/commons/7/7c/Kaggle_logo.png)


In this Kernel, I am going to focus on the last two years Kaggle survey dataset. I will use these responses to create a guideline for those, who are trying to break into the field of **Data Science.** 

The idea of this kernel is simple. First, we will give an overview of the responses through visualization. Then we will go into more depth with each answer. So, let's get started. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

## How to display full (non-truncated) datafrRumiame information in html when converting from pandas dataframe to html?
pd.set_option('display.max_colwidth', -1)
## How to display all the columns without truncating in between. 
pd.set_option('display.max_columns', 5000)

import os
print(os.listdir("../input"))

## ignores warning. 
import warnings
warnings.filterwarnings('ignore')
# Any results you write to the current directory are saved as output.

# 2017
response_2017 = pd.read_csv("../input/kaggle-survey-2017/multipleChoiceResponses.csv", encoding='ISO-8859-1')
conversion_ration_2017 = pd.read_csv("../input/kaggle-survey-2017/conversionRates.csv")
schema_2017 = pd.read_csv("../input/kaggle-survey-2017/schema.csv")
free_from_response_2017 = pd.read_csv("../input/kaggle-survey-2017/freeformResponses.csv")

#2018
response_2018 = pd.read_csv("../input/kaggle-survey-2018/multipleChoiceResponses.csv")
schema_2018 = pd.read_csv("../input/kaggle-survey-2018/SurveySchema.csv")
free_from_responses_2018 = pd.read_csv("../input/kaggle-survey-2018/freeFormResponses.csv")

# 2019
response_2019 = pd.read_csv("../input/kaggle-survey-2019/multiple_choice_responses.csv")
questions_2019 = pd.read_csv("../input/kaggle-survey-2019/questions_only.csv")
schema_2019 = pd.read_csv("../input/kaggle-survey-2019/survey_schema.csv")
other_text_response_2019 = pd.read_csv("../input/kaggle-survey-2019/other_text_responses.csv")

In [ ]:
pd.options.display.max_columns = 999

pd.set_option('display.max_colwidth', -1)

# Exploratory Data Analysis(EDA)
***
## What are the Male/Female ratio?

In [ ]:
Dont delete this line as the names were changed. 
# response_2017 = multiple_choice_response_2017
# response_2018 = multiple_choice_response

In [ ]:
response_2017.GenderSelect.replace(['A different identity', 'Non-binary, genderqueer, or gender non-conforming'], 'other', inplace=True)
response_2018.Q1.replace(["Prefer not to say", "Prefer to self-describe"], "other", inplace=True)
response_2019.Q2.replace(["Prefer not to say", "Prefer to self-describe"], "other", inplace=True)

temp1 = response_2017.GenderSelect.value_counts()

temp2 = response_2018.Q1.value_counts()
temp2 = temp2[:len(temp2)-1]

temp3 = response_2019.Q2.value_counts()
temp3 = temp3[:len(temp3)-1]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# labels = temp1.keys()

fig = make_subplots(1, 3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['2017', '2018', '2019'])
fig.add_trace(go.Pie(labels=temp1.keys(), values=temp1.values, scalegroup='one',
                     name="2017"), 1, 1)
fig.add_trace(go.Pie(labels=temp2.keys(), values=temp2.values, scalegroup='one',
                     name="2018"), 1, 2)
fig.add_trace(go.Pie(labels=temp3.keys(), values=temp3.values, scalegroup='one',
                     name="2019"), 1, 3)

fig.update_layout(title_text='Kaggle Surveys(2017-2019)')
fig.show()

Even though the participation rates increased in Kaggle surveys, the ratio between male and female stayed almost the same. 

## Top Ten Countries with Most Respondents?

In [ ]:
response_2017.Country.replace("People 's Republic of China", "China", inplace=True)

response_2018.Q3.replace("United States of America", "United States", inplace=True)
response_2018.Q3.replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom", inplace=True)

response_2019.Q3.replace("United States of America", "United States", inplace=True)
response_2019.Q3.replace("People 's Republic of China", "China", inplace=True)
response_2019.Q3.replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom", inplace=True)

In [ ]:
def remove_text_implement_val_counts(df, feature, text = 'Other'):
    df = df.loc[:feature][df.loc[:feature] != text]
    df = df[feature].value_counts()
    return df

temp9 = remove_text_implement_val_counts(response_2019, "Q3", "Other") 
temp8 = remove_text_implement_val_counts(response_2018, "Q3") # default text is "Other" so there is no need to specify
temp7 = remove_text_implement_val_counts(response_2017, "Country")

tempdf = pd.concat([temp7, temp8, temp9], axis=1, join = 'inner',)
tempdf.columns = ['2017', '2018', '2019']

In [ ]:
import plotly.graph_objects as go
countries = tempdf.index

fig = go.Figure(data=[
    go.Bar(name='2017', x=countries, y=tempdf['2017'].values),
    go.Bar(name='2018', x=countries, y=tempdf['2018'].values), 
    go.Bar(name='2019', x=countries, y=tempdf['2019'].values)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

It looks like United States and India is participating the most, What took my attention is when I looked at China. The participation rate massively incresed during the year 2018 and went down in 2019. I am guessing China has already made their own version of Kaggle somewhere in the web. 

Most respondents are from **United States, India and China.** However, it looks like the amount of participation from these countries decreases in 2018. 

## What are their ages?

In [ ]:
## creating age bins for 2017 survey according to age groups done by 2018 survey, so that we can compare side by side. 
def age_group(age):
    a = ''
    if age <= 21:
        a = '18-21'
    elif age <= 24: 
        a = '22-24'
    elif age <= 29:
        a = '25-29'
    elif age <= 34:
        a = '30-34'
    elif age <= 39:
        a = '35-39'
    elif age <= 44:
        a = '40-44'
    elif age <= 49:
        a = '45-49'
    elif age <= 54:
        a = '50-54'
    elif age <= 59:
        a = '55-59'
    elif age <= 69:
        a = '60-69'
    elif age <= 79:
        a = '70-79'
    else:
        a = '80+'
    return a
## creating a new columns Age_bins and applying "age_group" function. 
response_2017['Age_bins'] = response_2017.Age.map(age_group)

In [ ]:
##2017 age value_counts()
temp = pd.DataFrame(response_2017.Age_bins.value_counts())

##2018 age value_counts()
temp1 = pd.DataFrame(response_2018.Q2.value_counts())
temp1 = temp1[:len(temp1)-1]

##2019 age value_counts()
temp2 = pd.DataFrame(response_2019.Q1.value_counts())
temp2 = temp2[:len(temp2)-1]

## merging 2017 & 2018
temp = temp.merge(temp1, how = 'inner', left_index=True, right_index=True)
temp = temp.merge(temp2, how = 'inner', left_index=True, right_index=True)


temp.columns = ['2017','2018', '2019']

In [ ]:
x = temp.index.tolist()
y7 = temp["2017"].tolist()
y8 = temp["2018"].tolist()
y9 = temp["2019"].tolist()

trace1 = go.Bar(
    x=x,
    y=y7,
    name="2017",
    #text=y,
    textposition = 'auto',
    marker=dict(
        color='rgb(0,0,128)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6
)

trace2 = go.Bar(
    x=x,
    y=y8,
    name = '2018',
    #text=y2,
    textposition = 'auto',
    marker=dict(
        color='rgb(0,128,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.6
)
trace3 = go.Bar(
    x=x,
    y=y9,
    name = '2019',
    #text=y2,
    textposition = 'auto',
    marker=dict(
        color='rgb(0,128,105)',
        line=dict(
            color='rgb(8,48,205)',
            width=1.5),
        ),
    opacity=0.6
)


data = [trace1,trace2, trace3]
layout= go.Layout(
    title ='Age Groups of The Respondents (2017-2018)',

)
fig = go.Figure(data=data, layout=layout)
fig['layout']['xaxis'].update(title='Age group')
fig['layout']['yaxis'].update(title='Count')

fig.show()

Most respondents are within the range from **18 to 39 years of age.** It seems like this is the perfect age to invest your time towards learning data science. 

## What Certificates do they hold?

In [ ]:
temp = response_2017[response_2017.FormalEducation != 'I prefer not to answer']
temp.FormalEducation.replace('I did not complete any formal education past high school', 'No formal education past high school', inplace=True)
temp.FormalEducation.replace("Master's degree", "Master’s degree", inplace=True)
temp.FormalEducation.replace("Bachelor's degree", "Bachelor’s degree", inplace=True)
temp.FormalEducation.replace("Some college/university study without earning a bachelor's degree", "Some college/university study without earning a bachelor’s degree", inplace=True)
temp = temp.FormalEducation.value_counts()



temp1 = response_2018[response_2018.Q4 != 'I prefer not to answer']
temp1 = temp1.Q4.value_counts()
temp1 = temp1[:len(temp1)-1]
temp1

temp = pd.DataFrame(temp).merge(pd.DataFrame(temp1), how = 'inner', left_index = True, right_index = True)
temp.columns = ['2017','2018']

In [ ]:
fig = {
  "data": [
    {
      "values": temp['2017'].values.tolist(),#
      "labels": temp.index.tolist(),
      "domain": {"x": [0, .48]},
      "name": "in 2017",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie",
         "marker": {"line":{"color":'#000000', "width":2}} 
    },
    {
      "values": temp['2018'].values.tolist(), 
      "labels": temp.index.tolist(),
      #"text":["CO2"],
      "textposition":"inside",
      "domain": {"x": [.52, 1]},
      "name": "in 2018",
      "hoverinfo":"percent+label+name",
      "hole": .4,
      "type": "pie",
         "marker": {"line":{"color":'#000000', "width":2}} 
        
    }],
  "layout": {
        "title":"Male & Female Ratio(2017-2018)",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "2017",
                "x": 0.225,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "2018",
                "x": 0.775,
                "y": 0.5
            }
        ]
    }
    
}

fig.show()

Most respondents hold at least a Master's degree. 

## What was their Major?

In [ ]:
response_2018.Q5.replace(['Environmental science or geology','A business discipline (accounting, economics, finance, etc.)'], 'Other', inplace=True)

response_2017.MajorSelect.replace(['Management information systems','A health science'],'Other', inplace=True)

response_2017.MajorSelect.replace('Electrical Engineering','Engineering (non-computer focused)', inplace=True)

response_2018.Q5.replace('Computer science (software engineering, etc.)', 'Computer Science', inplace=True)

response_2017.MajorSelect.replace('A humanities discipline','Humanities (history, literature, philosophy, etc.)', inplace=True)

response_2017.MajorSelect.replace('Physics', 'Physics or astronomy', inplace=True)

response_2017.MajorSelect.replace('Biology', 'Medical or life sciences (biology, chemistry, medicine, etc.)', inplace=True)

response_2017.MajorSelect.replace(['A social science','Psychology'], 'Social sciences (anthropology, psychology, sociology, etc.)', inplace=True)

In [ ]:
temp = response_2018.Q5.value_counts(dropna=False)
temp = temp[:len(temp)-1]

temp1 = response_2017.MajorSelect.value_counts()

temp = pd.DataFrame(temp).merge(pd.DataFrame(temp1), how = 'inner', left_index = True, right_index = True)
temp.columns = ['2018','2017']

In [ ]:
temp = temp.reindex([i for i in reversed(temp.index)])
trace1 = go.Bar(
    y=temp.index.tolist(),
    x=temp['2017'].tolist(),
    name='2017',
    orientation = 'h',
    marker = dict(
        color = 'rgb(0,0,128)',
        line = dict(
            color = 'rgb(0,128,100)',
            width = 3)
    )
)
trace2 = go.Bar(
    y=temp.index.tolist(),
    x=temp['2018'].tolist(),
    name='2018',
    orientation = 'h',
    marker = dict(
        color = 'rgb(0,128,255)',
        line = dict(
            color = 'rgb(0,128,100)',
            width = 3)
    )
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group', 
    margin=go.layout.Margin(l = 400)
)

fig = go.Figure(data=data, layout=layout)
fig.layout.update(height = 700)
fig.show()

It looks like most of the respondents have come from Computer Science, Math or Engineering background.

## What is Their Title?

In [ ]:
## Importing Necessary modules. 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from PIL import Image

## Getting the image. 
wine_mask = np.array(Image.open("../input/png-files3/paw.png"))

##
def transform_format(val):
    if val == 0:
        return 255
    else:
        return val
# Transform your mask into a new one that will work with the function:
transformed_wine_mask = np.ndarray((wine_mask.shape[0],wine_mask.shape[1]), np.int32)

for i in range(len(wine_mask)):
    transformed_wine_mask[i] = list(map(transform_format, wine_mask[i]))


#print(names)
names = response_2018["Q6"][~pd.isnull(response_2018["Q6"])]

# Create a word cloud image
wc = WordCloud(background_color="white",
               colormap = 'Blues',
               #max_font_size=30, 
               #height = 1200,
               #max_words=1000, 
              # min_font_size=12,
               mask=transformed_wine_mask,
               stopwords=STOPWORDS,
               contour_width=2, 
               contour_color='blue')

# Generate a wordcloud
wc.generate(' '.join(names))   



## show
plt.figure(figsize=(20,20))
plt.imshow(wc, interpolation='bilinear')
plt.title("What is Their Title?", fontsize=35)
plt.axis("off")
plt.show() 

For those who are not familier with wordcloud like the one above, it is used to find out frequiently used terms in any text. In this case, we are using it to find the frequiently used titles given by survey takers. According to this wordcloud above, most of the respondents are either **Data Scientist** or **Software Engineer**

## In What Industries do They Work??

In [ ]:
#print(names)
names = response_2018["Q7"][~pd.isnull(response_2018["Q7"])]

# Creating mask
mask = np.array(Image.open("../input/png-files3/U2Epn.png")) 

## Generating wordcloud
wordcloud_usa = WordCloud(stopwords=STOPWORDS, 
                          background_color="white", 
                          colormap = 'Blues_r',
                          #mode="RGB", 
                          #max_words=1000, 
                          mask=mask).generate(' '.join(names))

# create coloring from image
#image_colors = ImageColorGenerator(mask)
plt.figure(figsize=[20,20])
plt.imshow(wordcloud_usa, interpolation="bilinear")
plt.axis("off")
plt.title("In What Industries do They Work?", fontsize = 40)
plt.show()


Most of the survey takers work in the <b>Computers Technology</b> industry. There are also others who work in the Academic, Education, Accounting and Finance. 

## How much do they earn?

In [ ]:
## creating df's to analyze responses only from the professionals. 
professionals_df_18 = response_2018[(response_2018.Q6 != 'Student') & (response_2018.Q7 != 'I am a student')]
professionals_df_17 = response_2017[(response_2017.EmploymentStatus != 'Not employed, but looking for work') & (response_2017.EmploymentStatus != 'I prefer not to say')]

In [ ]:
temp = professionals_df_18[(professionals_df_18.Q9 != 'What is your current yearly compensation (approximate $USD)?') & 
                         (professionals_df_18.Q9 != 'I do not wish to disclose my approximate yearly compensation')]
temp.Q9.dropna(inplace=True)

## Writing a function to determine the average. 
def temp_avg_fun(c):
    a = c.split("-")[0]
    a = a.strip('+')
    a = a.split(",")[0]
    a = int(a+str('000'))
    try:
        b = c.split("-")[1]
    except:
        b = '500,000'
    b = b.split(',')[0]
    b = int(b+str('000'))
    d = (a+b)/2
    return d

## Creating a new column with avg_salary. 
temp['Avg_salary'] = temp.Q9.apply(lambda x: temp_avg_fun(x))

##getting rid of "Other" from the country('Q3') column. 
temp = temp[temp.Q3 != 'Other']

In [ ]:
## Getting the top 10 countries. 
temp_df = pd.DataFrame(temp.Q3.value_counts())
##Changing the column names. 
temp_df.columns = ['total']
## Getting percentage


In [ ]:
temp_df['percent'] = temp_df.total.apply(lambda x: round(x/sum(temp_df.total)*100,2))

temp_df = temp_df.merge(temp.pivot_table(values = 'Avg_salary', index='Q3').reset_index(),left_index=True, right_on='Q3', how='left')

temp_df.sort_values(by = 'percent', inplace=True, ascending=False)

temp_df = temp_df.head(10)

temp_df.sort_values(by = 'percent', inplace=True, ascending=True)

In [ ]:
y_saving = temp_df.percent.tolist()
y_net_worth = temp_df.Avg_salary.tolist()
x_saving = temp_df.Q3.tolist()
x_net_worth = temp_df.Q3.tolist()
trace0 = go.Bar(
    x=y_saving,
    y=x_saving,
    marker=dict(
        color='rgb(0,0,128)',
        line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1),
    ),
    name='Percentage of the Respondent Countries',
    orientation='h',
)
trace1 = go.Scatter(
    x=y_net_worth,
    y=x_net_worth,
    mode='lines+markers',
    line=dict(
        color='rgb(0,0, 205)'),
    name='Average Salary',
)
layout = dict(
    title='Percentage of the Respondents Countries and Average Salary',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.85],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=True,
        showticklabels=False,
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=2,
        domain=[0, 0.85],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.42],
    ),
    xaxis2=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0.47, 1],
        side='top',
        dtick=25000,
    ),
    legend=dict(
        x=0.029,
        y=1.038,
        font=dict(
            size=10,
        ),
    ),
    margin=dict(
        l=100,
        r=20,
        t=70,
        b=70,
    ),
    paper_bgcolor='rgb(152, 204, 255)',
    plot_bgcolor='rgb(152, 204, 255)',
)

annotations = []

y_s = np.round(y_saving, decimals=2)
y_nw = np.rint(y_net_worth)

# Adding labels
for ydn, yd, xd in zip(y_nw, y_s, x_saving):
    # labeling the scatter savings
    annotations.append(dict(xref='x2', yref='y2',
                            y=xd, x=ydn - 20000,
                            text='{:,}'.format(ydn) + 'M',
                            font=dict(family='Arial', size=12,
                                      color='rgb(0, 0, 205)'),
                            showarrow=False))
    # labeling the bar net worth
    annotations.append(dict(xref='x1', yref='y1',
                            y=xd, x=yd + 3,
                            text=str(yd) + '%',
                            font=dict(family='Arial', size=12,
                                      color='rgb(0, 0, 128)'),
                            showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.2, y=-0.109,
                       # text='OECD "' +
                       #      '(2015), Household savings (indicator), ' +
                       #      'Household net worth (indicator). doi: ' +
                        #     '10.1787/cfc6f499-en (Accessed on 05 June 2015)',
                        font=dict(family='Arial', size=10,
                                  color='rgb(150,150,150)'),
                        showarrow=False))

layout['annotations'] = annotations

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001)

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

fig['layout'].update(layout)
fig.show()

It is true that <b> the United States, India & China</b> are the three leading countries in the tech industry. If you are following the latest trends you would know that China is going all out and trying to dominate the tech industry. However, with all that being said, the market for US citizens is still a lot better than other countries as the chart above shows. 

## How long does it take to learn Data Science?

In [ ]:
trace = go.Bar(
    x = professionals_df_17.LearningDataScienceTime.value_counts().index,
    y = professionals_df_17.LearningDataScienceTime.value_counts().values,
    opacity=0.5,
    marker=dict(
        color='rgba(152,204,255,0.9)',
        line=dict(
            color='rgba(0,0,0,0.5)',
            width=1.5,
        )
    
),
) 

layout= go.Layout(
    title ='How long does it take to learn Data Science?',

)
data = [trace]
fig = go.Figure(data=data, layout=layout)
fig['layout']['xaxis'].update(title='Age group')
fig['layout']['yaxis'].update(title='Response Count')
fig['layout'].update(paper_bgcolor='rgb(255,255,255)')
fig.show()

According to most professionals, it takes <b> less than a year to learn Data Science.</b>

## What are the Recommended Choice of Machine Learning Tools?

In [ ]:

trace = go.Bar(
    x = professionals_df_17.MLToolNextYearSelect.value_counts().index,
    y = professionals_df_17.MLToolNextYearSelect.value_counts().values,
    opacity=0.5,
    marker=dict(
        color='rgba(152,204,255,0.9)',
        line=dict(
            color='rgba(0,0,0,0.5)',
            width=1.5,
        )
    
),
) 

layout= go.Layout(
    title ='Recommended Choice of Machine Learning Tools',
    margin=go.layout.Margin(b = 400)

)
data = [trace]
fig = go.Figure(data=data, layout=layout)
fig['layout']['xaxis'].update(title='Machine Learning Tools')
fig['layout']['yaxis'].update(title='Response Count')
fig['layout'].update(paper_bgcolor='rgb(255,255,255)')
fig.layout.update(height = 1000)
fig.show()

## What are the Recommended Choice of Machine Learning Methods?

In [ ]:


trace = go.Bar(
    x = professionals_df_17.MLMethodNextYearSelect.value_counts().index,
    y = professionals_df_17.MLMethodNextYearSelect.value_counts().values,
    opacity=0.5,
    marker=dict(
        color='rgba(152,204,255,0.9)',
        line=dict(
            color='rgba(0,0,0,0.5)',
            width=1.5,
        )
    
),
) 

layout= go.Layout(
    title ='Recommended Choice of Machine Learning Methods',
    margin=go.layout.Margin(b = 222)

)
data = [trace]
fig = go.Figure(data=data, layout=layout)
fig['layout']['xaxis'].update(title='Machine Learning Methods')
fig['layout']['yaxis'].update(title='Response Count')
fig['layout'].update(paper_bgcolor='rgb(255,255,255)')
fig.layout.update(height = 1000)
fig.show()

## What are the Recommended Choice of Languages?

In [ ]:
trace = go.Bar(
    x = professionals_df_17.LanguageRecommendationSelect.value_counts().index,
    y = professionals_df_17.LanguageRecommendationSelect.value_counts().values,
    opacity=0.5,
    marker=dict(
        color='rgba(152,204,255,0.9)',
        line=dict(
            color='rgba(0,0,0,0.5)',
            width=1.5,
        )
    
),
) 

layout= go.Layout(
    title ='Recommended Choice of Languages',
    #margin=go.layout.Margin(b = 222)

)
data = [trace]
fig = go.Figure(data=data, layout=layout)
fig['layout']['xaxis'].update(title='Languages')
fig['layout']['yaxis'].update(title='Response Count')
fig['layout'].update(paper_bgcolor='rgb(255,255,255)')
fig.show()

## What do Professionals Think are the Most Important Job Skills?

In [ ]:
# getting only the working columns
temp_df = professionals_df_17.loc[:,"JobSkillImportanceBigData":"JobSkillImportanceVisualizations"]
#temp = temp_df.JobSkillImportanceBigData.value_counts(dropna=True)
#temp = pd.DataFrame(temp)
#temp.columns=['BigData']
#temp.BigData = temp.BigData.apply(lambda x: round(x/sum(temp.BigData)*100,0))

## writing to function to get the percentage of a column, round it and then returning a df with that rounded percentage. 
def find_percentage(df, col):
    return pd.DataFrame(round(df[col].value_counts()/df[col].value_counts().sum()*100,1))


## 
count = 0
for i in temp_df.columns:
    if count == 0:
        temp = find_percentage(temp_df, str(i))
    else:
        temp1 = find_percentage(temp_df, str(i))
        temp = temp.merge(temp1, left_index=True, right_index=True, how='inner')
    count += 1

## custom ma
temp.columns = ['BigData','Degree','Statistics','Enterprise Tools','Python','R','SQL','Kaggle Ranking','Online Courses','Visualizations']

In [ ]:
temp.head()

In [ ]:
temp = temp.reindex(['Unnecessary','Nice to have', 'Necessary'])

## [much worse, slightly worse....much better]
## list of indexes
top_labels = temp.index.tolist()

colors = ['rgba(135,206,250,0.8)', 'rgba(0,191,255, 0.8)','rgba(30,144,255, 0.8)','rgba(0,0,255,0.8)','rgba(0,0,128,0.8)']


#[percents]
## creating a list of lists with x data's
x_data = [temp[i].tolist() for i in temp.columns]

# y axis
## creating a list of column names
#y_data = ['Online', 'Bootcamp']
y_data = temp.columns.tolist()

traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(224, 255, 255)',
    plot_bgcolor='rgb(224, 255, 255)',
    margin=dict(
        l=120,
        r=10,
        t=140,
        b=80
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=12,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
fig.show()

It looks like the two most important skills are <b>Python & Statistics.</b> There is also a pretty good emphesis on **BigData** & **Visualixation.** Contrary to popular belief, most professionals don't think having a **degree** is neceassary and some **13%** thinks its unnecessary. 

## What do Professionals think about learning Online/BootCamp vs Traditional Brick and mortar institutions?

In [ ]:
professionals_df_18.Q39_Part_1.replace('Neither better nor worse', "Neutral", inplace=True)
professionals_df_18.Q39_Part_1.replace('No opinion; I do not know', "Neutral", inplace=True)
professionals_df_18.Q39_Part_2.replace('No opinion; I do not know', "Neutral", inplace=True)
professionals_df_18.Q39_Part_2.replace('Neither better nor worse', "Neutral", inplace=True)

## Online learning platforms and MOOCs:
temp = professionals_df_18.Q39_Part_1.value_counts()
temp = temp[:len(temp)-1]
temp = pd.DataFrame(temp,)

## - In-person bootcamps:
temp1 = professionals_df_18.Q39_Part_2.value_counts()
temp1 = temp1[:len(temp1)-1]
temp1 = pd.DataFrame(temp1)

## merging both dataframes. 
temp = temp.merge(temp1 ,left_index=True, right_index=True, how = 'inner')
temp.columns = ['online_total', 'bootCamp_total']

## format the index as I like. 
indexes = ["Much worse", "Slightly worse", "Neutral", "Slightly better", "Much better"]
## rearrange indexes
temp = temp.reindex(indexes)  


temp['online_percent'] = round(temp.online_total/sum(temp.online_total)*100,0).apply(lambda x: int(x))
temp['bootcamp_percent'] = round(temp.bootCamp_total/sum(temp.online_total)*100,0).apply(lambda x: int(x))

#temp.online_percent = temp.online_percent.apply(lambda x: int(x))
#temp.bootcamp_percent = temp.bootcamp_percent.apply(lambda x: int(x))

In [ ]:
## [much worse, slightly worse....much better]
top_labels = temp.index.tolist()

colors = ['rgba(135,206,250,0.8)', 'rgba(0,191,255, 0.8)','rgba(30,144,255, 0.8)','rgba(0,0,255,0.8)','rgba(0,0,128,0.8)','rgba(0,0,128,0.8)' ]



#[percents]
x_data = [temp.online_percent.tolist(),
          temp.bootcamp_percent.tolist()]

# y axis
y_data = ['Online', 'Bootcamp']


traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(224, 255, 255)',
    plot_bgcolor='rgb(224, 255, 255)',
    margin=dict(
        l=120,
        r=10,
        t=140,
        b=80
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=12,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=True))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
fig.show()


Though most professionals stayed neutral, As many as almost half of them thinks it is either slightly better or much better to take a **coding boot camp** or **online courses** than getting a degree from traditional schools. 

## What are the Key Factors in a Job Environment?

In [ ]:
temp_df = professionals_df_17.loc[:,"JobFactorLearning":"JobFactorPublishingOpportunity"]

count = 0
for i in temp_df.columns:
    if count == 0:
        temp = find_percentage(temp_df, str(i))
    else:
        temp1 = find_percentage(temp_df, str(i))
        temp = temp.merge(temp1, left_index=True, right_index=True, how='inner')
    count += 1

temp.columns = ['Learning','Salary','Office','Languages','Commute','Management','Experience Level','Department','Title','Company Funding','Impact','Remote','Industry','Leader Reputation','Diversity','Publishing Opportunity']


In [ ]:
temp = temp.reindex(['Not important','Somewhat important', 'Very Important'])

## [much worse, slightly worse....much better]
## list of indexes
top_labels = temp.index.tolist()
colors = ['rgba(135,206,250,0.8)', 'rgba(0,191,255, 0.8)','rgba(30,144,255, 0.8)','rgba(0,0,255,0.8)','rgba(0,0,128,0.8)','rgba(0,0,128,0.8)' ]

#[percents]
## creating a list of lists with x data's
x_data = [temp[i].tolist() for i in temp.columns]

# y axis
## creating a list of column names
#y_data = ['Online', 'Bootcamp']
y_data = temp.columns.tolist()

traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(224, 255, 255)',
    plot_bgcolor='rgb(224, 255, 255)',
    margin=dict(
        l=120,
        r=10,
        t=140,
        b=80
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=10,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.05,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.05,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
fig.layout.update(height = 800)
fig.show()

According to the professionals, the most important aspects of a job is **learning** besides **office environment and salary**. So, if you are in a job that you are not learning anything; it is probably a good idea to look for a change. 

## What are the Frequently used Skills/Tools?

In [ ]:
temp_df = professionals_df_17.loc[:,"WorkToolsFrequencyAmazonML":"WorkToolsFrequencyUnix"]

count = 0
for i in temp_df.columns:
    if count == 0:
        temp = find_percentage(temp_df, str(i))
    else:
        temp1 = find_percentage(temp_df, str(i))
        temp = temp.merge(temp1, left_index=True, right_index=True, how='inner')
    count += 1

temp.columns = ['AmazonML','AWS','Angoss','C','Cloudera','DataRobot','Flume','GCP','Hadoop','IBMCognos','IBMSPSSModeler','IBMSPSSStatistics','IBMWatson','Impala','Java','Julia','Jupyter','KNIMECommercial','KNIMEFree','Mathematica','MATLAB','Azure','Excel','MicrosoftRServer','MicrosoftSQL','Minitab','NoSQL','Oracle','Orange','Perl','Python','Qlik','R','RapidMinerCommercial','RapidMinerFree','Salfrod','SAPBusinessObjects','SASBase','SASEnterprise','SASJMP','Spark','SQL','Stan','Statistica','Tableau','TensorFlow','TIBCO','Unix']

In [ ]:
temp = temp.reindex(['Rarely','Sometimes', 'Often','Most of the time'])

## [much worse, slightly worse....much better]
## list of indexes
top_labels = temp.index.tolist()

colors = ['rgba(135,206,250,0.8)', 'rgba(0,191,255, 0.8)','rgba(30,144,255, 0.8)','rgba(0,0,255,0.8)','rgba(0,0,128,0.8)']


#[percents]
## creating a list of lists with x data's
x_data = [temp[i].tolist() for i in temp.columns]

# y axis
## creating a list of column names
#y_data = ['Online', 'Bootcamp']
y_data = temp.columns.tolist()

traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(224, 255, 255)',
    plot_bgcolor='rgb(224, 255, 255)',
    margin=dict(
        l=80,
        r=10,
        t=120,
        b=30
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.02,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.02,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
fig['layout'].update(height = 1300);
iplot(fig, filename='bar-colorscale')

It looks like **Python, SQL, Unix and Jupyter Notebook** are some of the most used tools/skills based on the survey given by professionals. 

## What are Some of the Challenges Companies Deal With?

In [ ]:
temp_df = professionals_df_17.loc[:,'WorkChallengeFrequencyPolitics':'WorkChallengeFrequencyDataAccess']

count = 0
for i in temp_df.columns:
    if count == 0:
        temp = find_percentage(temp_df, str(i))
    else:
        temp1 = find_percentage(temp_df, str(i))
        temp = temp.merge(temp1, left_index=True, right_index=True, how='inner')
    count += 1
    
temp.columns = ['Politics','Unused Results','Unuseful Instrumenting','Deployment','Dirty Data','Explaining','Pass','Integration','Talent','Data Funds','Domain Expertise','ML','Tools','Expectations','IT Coordination','Hiring Funds','Privacy','Scaling','Environments','Clarity','DataAccess']

In [ ]:
temp = temp.reindex(['Rarely','Sometimes', 'Often','Most of the time'])

## [much worse, slightly worse....much better]
## list of indexes
top_labels = temp.index.tolist()
colors = ['rgba(135,206,250,0.8)', 'rgba(0,191,255, 0.8)','rgba(30,144,255, 0.8)','rgba(0,0,255,0.8)','rgba(0,0,128,0.8)']

#[percents]
## creating a list of lists with x data's
x_data = [temp[i].tolist() for i in temp.columns]

# y axis
## creating a list of column names
#y_data = ['Online', 'Bootcamp']
y_data = temp.columns.tolist()

traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(224, 255, 255)',
    plot_bgcolor='rgb(224, 255, 255)',
    margin=dict(
        l=80,
        r=10,
        t=120,
        b=30
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=10,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.02,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.02,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
fig['layout'].update(height = 1300);
iplot(fig, filename='bar-colorscale')

Hmm.. .   . <b>Dirty data and Hiring funds</b> seems to be the top challenging aspects that companies deal with. 

## What are the Frequently used Methods in the Work Place?

In [ ]:
temp_df = professionals_df_17.loc[:,'WorkMethodsFrequencyA/B':'WorkMethodsFrequencyTimeSeriesAnalysis']

count = 0
for i in temp_df.columns:
    if count == 0:
        temp = find_percentage(temp_df, str(i))
    else:
        temp1 = find_percentage(temp_df, str(i))
        temp = temp.merge(temp1, left_index=True, right_index=True, how='inner')
    count += 1
    
temp.columns = ['A/B','Association Rules','Bayesian','CNNs','Collaborative Filtering','Cross Validation','Data Visualization','Decision Trees','Ensemble Methods','Evolutionary Approaches','GANs','GBM','HMMs','KNN','LiftAnalysis','Logistic Regression','MLN','Naive Bayes','NLP','Neural Networks','PCA','Prescriptive Modeling','Random Forests','Recommender Systems','RNNs','Segmentation','Simulation','SVMs','Text Analysis','Time Series Analysis']

In [ ]:
temp = temp.reindex(['Rarely','Sometimes', 'Often','Most of the time'])

## [much worse, slightly worse....much better]
## list of indexes
top_labels = temp.index.tolist()
colors = ['rgba(135,206,250,0.8)', 'rgba(0,191,255, 0.8)','rgba(30,144,255, 0.8)','rgba(0,0,255,0.8)','rgba(0,0,128,0.8)']

#[percents]
## creating a list of lists with x data's
x_data = [temp[i].tolist() for i in temp.columns]

# y axis
## creating a list of column names
#y_data = ['Online', 'Bootcamp']
y_data = temp.columns.tolist()

traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(224, 255, 255)',
    plot_bgcolor='rgb(224, 255, 255)',
    margin=dict(
        l=80,
        r=10,
        t=120,
        b=30
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=10,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.02,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.02,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
fig['layout'].update(height = 1300);
iplot(fig, filename='bar-colorscale')

<b>Data Visualization, Cross Validation, NLP and Neural Networks</b> are some of the most used methods in the tech industry.

# Credits

* All thanks to [Plotly](https://plot.ly/python/) for publishing well documented data visualization resources. 
* Thanks to Kaggle for being an incredible host and providing these amazing datasets. 
* And most of all, thanks to Kaggle community. It's been an amazing ride. 

<div>
*** This kernel will always be a work in progress. I will incorporate more charts with each update. If you have any idea/suggestions about this notebook, please let me know. Any feedback about further improvements would be genuinely appreciated.***

***
If you like to discuss any other projects or have a chat about data science topics, I'll be more than happy to connect with you on:

**LinkedIn:** https://www.linkedin.com/in/masumrumi/ 

**My Website:** http://masumrumi.com/ 


### If you have come this far, Congratulations!!

### If this notebook helped you in any way or you liked it, please upvote and/or leave a comment!! :) 
***

In [0]:
import pandas as pd
multiple_choice_responses = pd.read_csv("../input/kaggle-survey-2019/multiple_choice_responses.csv")
other_text_responses = pd.read_csv("../input/kaggle-survey-2019/other_text_responses.csv")
questions_only = pd.read_csv("../input/kaggle-survey-2019/questions_only.csv")
survey_schema = pd.read_csv("../input/kaggle-survey-2019/survey_schema.csv")